In [ ]:

import torch
print(f"GPU: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device: {torch.cuda.get_device_name(0)}")


GPU: True
Device: Tesla T4


In [ ]:

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install pandas scikit-learn matplotlib tqdm pillow seaborn timm

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:

import os
import zipfile
from pathlib import Path

DATA_ROOT = Path('/content/data')
DATA_ROOT.mkdir(parents=True, exist_ok=True)

urls = {
    'train_images': 'https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Training_Input.zip',
    'train_labels': 'https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Training_GroundTruth.zip',
    'val_images': 'https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Validation_Input.zip',
    'val_labels': 'https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Validation_GroundTruth.zip'
}

expected = {
    'train_images': DATA_ROOT / 'ISIC2018_Task3_Training_Input',
    'train_labels': DATA_ROOT / 'ISIC2018_Task3_Training_GroundTruth' / 'ISIC2018_Task3_Training_GroundTruth.csv',
    'val_images': DATA_ROOT / 'ISIC2018_Task3_Validation_Input',
    'val_labels': DATA_ROOT / 'ISIC2018_Task3_Validation_GroundTruth' / 'ISIC2018_Task3_Validation_GroundTruth.csv',
}

print("📦 Checking dataset presence...")
for key, url in urls.items():
    exp = expected[key]
    if (exp.is_dir() and any(exp.iterdir())) or (exp.is_file() and exp.exists()):
        print(f"✅ {key} already present, skipping download.")
        continue
    zip_path = DATA_ROOT / f'{key}.zip'
    print(f"📥 Downloading {key}...")
    !timeout 300 wget -c -q --tries=3 --timeout=30 -O "$zip_path" {url} || curl -L -o "$zip_path" --max-time 300 --retry 2 {url}
    if not zip_path.exists() or zip_path.stat().st_size < 1000:
        raise RuntimeError(f"Download failed for {key} from {url}")
    print(f"📂 Extracting {key}...")
    with zipfile.ZipFile(zip_path, 'r') as zf:
        zf.extractall(DATA_ROOT)
    zip_path.unlink()

print("✅ Dataset ready!")

📦 Checking dataset presence...
📥 Downloading train_images...
📂 Extracting train_images...
📥 Downloading train_labels...
📂 Extracting train_labels...
📥 Downloading val_images...
📂 Extracting val_images...
📥 Downloading val_labels...
📂 Extracting val_labels...
✅ Dataset ready!


In [ ]:

from pathlib import Path

class Config:
    BASE_DIR = Path('/content')
    DATA_ROOT = BASE_DIR / 'data'
    TRAINED_MODELS_DIR = BASE_DIR / 'trained_models'

    TRAIN_IMG_PATH = DATA_ROOT / "ISIC2018_Task3_Training_Input"
    TRAIN_CSV_PATH = DATA_ROOT / "ISIC2018_Task3_Training_GroundTruth" / "ISIC2018_Task3_Training_GroundTruth.csv"
    VAL_IMG_PATH = DATA_ROOT / "ISIC2018_Task3_Validation_Input"
    VAL_CSV_PATH = DATA_ROOT / "ISIC2018_Task3_Validation_GroundTruth" / "ISIC2018_Task3_Validation_GroundTruth.csv"

    MODELS = ['tf_efficientnetv2_s', 'convnext_tiny', 'swin_tiny_patch4_window7_224']
    NUM_CLASSES = 7
    CLASS_NAMES = ['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']

    IMAGE_SIZE = 224
    BATCH_SIZE = 32
    NUM_EPOCHS = 25
    LEARNING_RATE = 3e-4
    WEIGHT_DECAY = 1e-5
    EARLY_STOP_PATIENCE = 5

Config.TRAINED_MODELS_DIR.mkdir(exist_ok=True)

In [ ]:

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import pandas as pd
from PIL import Image
import timm
import torchvision.transforms as transforms
from tqdm import tqdm
import numpy as np
from pathlib import Path

class ISICDataset(Dataset):
    def __init__(self, img_dir, csv_path, transform=None, is_train=True):
        self.img_dir = Path(img_dir)
        self.transform = transform
        full_df = pd.read_csv(csv_path)

        full_df['label'] = full_df.iloc[:, 1:].idxmax(axis=1)

        if is_train:
            self.data_df = full_df.sample(frac=1, random_state=42).reset_index(drop=True)
            print("\nTraining dataset uses FULL set with imbalance-aware strategies (sampler + loss).")
        else:
            self.data_df = full_df

        self.labels = self._convert_labels()

        vc = self.get_counts()
        dist_str = ", ".join(f"{Config.CLASS_NAMES[i]}:{int(c)}" for i, c in enumerate(vc))
        print(f"Class distribution ({'train' if is_train else 'val'}): {dist_str}")

    def _convert_labels(self):
        class_map = {name: i for i, name in enumerate(Config.CLASS_NAMES)}
        return self.data_df['label'].map(class_map).values.astype(np.int64)

    def get_counts(self):
        counts = np.bincount(self.labels, minlength=Config.NUM_CLASSES)
        return counts

    def __len__(self): return len(self.data_df)

    def __getitem__(self, idx):
        img_name = self.data_df.iloc[idx]['image']
        img_path = self.img_dir / f"{img_name}.jpg"
        try:
            image = Image.open(img_path).convert('RGB')
        except:
            image = Image.new('RGB', (Config.IMAGE_SIZE, Config.IMAGE_SIZE), 'black')
        if self.transform: image = self.transform(image)
        return image, int(self.labels[idx])

# --- Model Architecture ---
class SkinCancerModel(nn.Module):
    def __init__(self, model_name, num_classes=Config.NUM_CLASSES):
        super().__init__()
        self.model_name = model_name
        self.backbone = timm.create_model(model_name, pretrained=True, num_classes=0)
        feature_dim = self.backbone.num_features
        self.classifier = nn.Sequential(
            nn.BatchNorm1d(feature_dim),
            nn.Dropout(0.5),
            nn.Linear(feature_dim, num_classes)
        )
    def forward(self, x): return self.classifier(self.backbone(x))


In [ ]:

import torch.optim as optim
from torch.cuda.amp import GradScaler, autocast
import time
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, WeightedRandomSampler
import torchvision.transforms as transforms

# --- Transforms ---
def get_transforms():
    train_transform = transforms.Compose([
        transforms.Resize((Config.IMAGE_SIZE, Config.IMAGE_SIZE)),
        transforms.RandomHorizontalFlip(0.5),
        transforms.RandomRotation(15),
        transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
    val_transform = transforms.Compose([
        transforms.Resize((Config.IMAGE_SIZE, Config.IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    return train_transform, val_transform
train_transform, val_transform = get_transforms()

# --- Class-Balanced Focal Loss 
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super().__init__()
        self.gamma = gamma
        self.reduction = reduction
        if alpha is not None:
            self.register_buffer('alpha', torch.as_tensor(alpha, dtype=torch.float32))
        else:
            self.alpha = None

    def forward(self, logits, targets):
        ce = nn.functional.cross_entropy(logits, targets, reduction='none')
        pt = torch.exp(-ce)
        loss = (1 - pt) ** self.gamma * ce
        if self.alpha is not None:
            at = self.alpha.gather(0, targets)
            loss = at * loss
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        return loss

def class_balanced_alpha(counts, beta=0.9999):
    effective_num = 1.0 - np.power(beta, counts)
    weights = (1.0 - beta) / np.clip(effective_num, 1e-8, None)
    weights = weights / weights.sum() * len(counts)
    return weights.astype(np.float32)

# --- Main Training Loop ---
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SESSION_START = time.time()

for model_name in Config.MODELS:
    if (time.time() - SESSION_START) / 3600 > 11.8:
        break
    print(f"\n{'='*60}\n▶️ Training: {model_name}\n{'='*60}")

    if (Config.TRAINED_MODELS_DIR / f"{model_name}_best.pth").exists():
        print(f"✅ Model already trained. Skipping."); continue

    train_dataset = ISICDataset(Config.TRAIN_IMG_PATH, Config.TRAIN_CSV_PATH, train_transform, is_train=True)
    val_dataset   = ISICDataset(Config.VAL_IMG_PATH,   Config.VAL_CSV_PATH,   val_transform,   is_train=False)

    # --- WeightedRandomSampler for balanced batches ---
    train_counts = train_dataset.get_counts()                          
    inv_freq = 1.0 / np.clip(train_counts, 1, None)                   
    sample_weights = inv_freq[train_dataset.labels]                    
    sample_weights = torch.as_tensor(sample_weights, dtype=torch.float32)
    sampler = WeightedRandomSampler(weights=sample_weights,
                                    num_samples=len(sample_weights),
                                    replacement=True)

    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE,
                              sampler=sampler, shuffle=False, num_workers=2)
    val_loader   = DataLoader(val_dataset,   batch_size=Config.BATCH_SIZE,
                              shuffle=False, num_workers=2)

    alpha = class_balanced_alpha(train_counts, beta=0.9999)
    loss_fn = FocalLoss(alpha=torch.tensor(alpha, dtype=torch.float32), gamma=2.0).to(DEVICE)

    model = SkinCancerModel(model_name).to(DEVICE)
    optimizer = optim.AdamW(model.parameters(), lr=Config.LEARNING_RATE, weight_decay=Config.WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10)
    scaler = GradScaler()
    best_acc = 0.0
    patience_counter = 0
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

    for epoch in range(Config.NUM_EPOCHS):
        model.train(); train_loss, train_correct, train_total = 0.0, 0, 0

        for data, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{Config.NUM_EPOCHS} [Train]"):
            data, targets = data.to(DEVICE), targets.to(DEVICE)
            optimizer.zero_grad(set_to_none=True)
            with autocast():
                outputs = model(data)
                loss = loss_fn(outputs, targets)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()
            _, p = outputs.max(1)
            train_total += targets.size(0)
            train_correct += p.eq(targets).sum().item()

        model.eval(); val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for data, targets in tqdm(val_loader, desc=f"Epoch {epoch+1}/{Config.NUM_EPOCHS} [Val]"):
                data, targets = data.to(DEVICE), targets.to(DEVICE)
                outputs = model(data)
                loss = loss_fn(outputs, targets)  # evaluate with same criterion
                val_loss += loss.item()
                _, p = outputs.max(1)
                val_total += targets.size(0)
                val_correct += p.eq(targets).sum().item()

        history['train_loss'].append(train_loss / max(1, len(train_loader)))
        history['train_acc'].append(100.0 * train_correct / max(1, train_total))
        history['val_loss'].append(val_loss / max(1, len(val_loader)))
        history['val_acc'].append(100.0 * val_correct / max(1, val_total))
        print(f"Train Acc: {history['train_acc'][-1]:.2f}% | Val Acc: {history['val_acc'][-1]:.2f}%")

        if history['val_acc'][-1] > best_acc:
            best_acc = history['val_acc'][-1]; patience_counter = 0
            torch.save({'model_state_dict': model.state_dict()}, Config.TRAINED_MODELS_DIR / f'{model_name}_best.pth')
            print(f"  🎯 New best accuracy: {best_acc:.2f}%")
        else:
            patience_counter += 1
            if patience_counter >= Config.EARLY_STOP_PATIENCE:
                print("⏹️ Early stopping triggered."); break

        scheduler.step()

print(f"\n🎉 Training session complete! Total time: {(time.time() - SESSION_START)/3600:.2f} hours")
print(f"✅ Models saved to: {Config.TRAINED_MODELS_DIR}")



▶️ Training: tf_efficientnetv2_s

Training dataset intelligently sampled for optimal performance.
Original size: 10015, New size: 5310
New class distribution:
 label
NV       2000
MEL      1113
BKL      1099
BCC       514
AKIEC     327
VASC      142
DF        115
Name: count, dtype: int64


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/86.5M [00:00<?, ?B/s]

/tmp/ipython-input-1070108076.py:49: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1/25 [Train]:   0%|          | 0/166 [00:00<?, ?it/s]/tmp/ipython-input-1070108076.py:59: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1/25 [Val]: 100%|██████████| 7/7 [00:01<00:00,  3.77it/s]


Train Acc: 56.25% | Val Acc: 75.13%
  🎯 New best accuracy: 75.13%


...

Train Acc: 87.51% | Val Acc: 82.90%


Epoch 11/25 [Val]: 100%|██████████| 7/7 [00:01<00:00,  3.85it/s]


Train Acc: 70.49% | Val Acc: 77.20%


Epoch 12/25 [Val]: 100%|██████████| 7/7 [00:01<00:00,  3.78it/s]

Train Acc: 72.56% | Val Acc: 78.24%
⏹️ Early stopping triggered.

🎉 Training session complete! Total time: 0.65 hours
✅ Models saved to: /content/trained_models


In [ ]:

from google.colab import files
from pathlib import Path
import time

print("="*50)
print("🚀 Starting Download of Trained Models...")
print("="*50)

MODELS_DIR = Path('/content/trained_models')

MODEL_NAMES = [
    "tf_efficientnetv2_s",
    "convnext_tiny",
    "swin_tiny_patch4_window7_224"
]

downloaded_count = 0
missing_files = []

for model_name in MODEL_NAMES:
    file_name = f"{model_name}_best.pth"
    file_path = MODELS_DIR / file_name

    print(f"\nChecking for: {file_name}...")

    if file_path.exists():
        print(f"✅ Found! Starting download for {file_name}. Please wait...")
        files.download(str(file_path))
        downloaded_count += 1
        time.sleep(2)
    else:
        print(f"⚠️ Not found. It might still be training or was stopped early.")
        missing_files.append(file_name)

print("\n" + "="*50)
print("📊 DOWNLOAD SUMMARY")
print("="*50)
print(f"Successfully downloaded {downloaded_count} model file(s).")

if missing_files:
    print("\nThe following models were not found:")
    for f_name in missing_files:
        print(f"  - {f_name}")

print("\n✅ Download process complete. You can now use your 'move_downloads.py' script on your local machine.")

🚀 Starting Download of Trained Models...

Checking for: tf_efficientnetv2_s_best.pth...
✅ Found! Starting download for tf_efficientnetv2_s_best.pth. Please wait...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Checking for: convnext_tiny_best.pth...
✅ Found! Starting download for convnext_tiny_best.pth. Please wait...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Checking for: swin_tiny_patch4_window7_224_best.pth...
✅ Found! Starting download for swin_tiny_patch4_window7_224_best.pth. Please wait...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


📊 DOWNLOAD SUMMARY
Successfully downloaded 3 model file(s).

✅ Download process complete. You can now use your 'move_downloads.py' script on your local machine.
